# 基于矩阵分解的评分预测

导入包。

In [ ]:
import numpy as np
import pandas as pd

读取评分数据集和电影数据集。

In [ ]:
ratings_data_set = pd.read_csv("data/ratings.csv").head(300)
movies_data_set = pd.read_csv("data/movies.csv")

将评分数据集划分为训练集、验证集和测试集，比例为 8:1:1。

In [ ]:
train_set = ratings_data_set.sample(frac=0.8, random_state=42)
validation_and_test_set = ratings_data_set.drop(train_set.index)
validation_set = validation_and_test_set.sample(frac=0.5, random_state=42)
test_set = validation_and_test_set.drop(validation_set.index)

确定评分矩阵的维度，即用户总数 m 和电影总数 n。

In [ ]:
users_num = ratings_data_set.userId.unique().shape[0]
movies_num = movies_data_set.movieId.unique().shape[0]

`build_R` 函数根据给定的评分数据，创建评分矩阵；未评分用 NaN 来表示。

In [ ]:
def build_R(ratings: pd.DataFrame):
    R = np.full((users_num, movies_num), np.nan)

    for rating in ratings.itertuples():
        row = rating.userId - 1
        column = movies_data_set[movies_data_set.movieId == rating.movieId].index[0]
        R[row, column] = rating.rating

    return R

`matrix_factorization` 函数使用 BiasSVD 算法来分解评分矩阵 R (m×n)，分解结果如下：

- P (m×k) 代表用户内在特征矩阵，可以理解为 m 位用户在 k 个特征方向上的喜好。
- Q (n×k) 代表电影内在特征矩阵，可以理解为 n 部电影在 k 个特征方向上的表现。
- B 代表全局偏置，即所有评分的平均值。
- BP (m×1) 代表用户偏置矩阵，即 m 位用户本身给分的高低倾向。
- BQ (n×1) 代表电影偏置矩阵，即 n 部电影本身质量对评分的影响。

In [ ]:
def matrix_factorization(R: np.ndarray, k=3, steps=3000, lr=0.0002, reg=0.01):
    m, n = R.shape

    P = np.random.rand(m, k)
    Q = np.random.rand(n, k)
    B = np.nanmean(R)
    BP = np.random.rand(m)
    BQ = np.random.rand(n)

    for _ in range(steps):
        for i, row in enumerate(R):
            for j, value in enumerate(row):
                if np.isnan(value):
                    continue

                diff = value - B - BP[i] - BQ[j] - np.dot(P[i], Q[j])
                P[i] += lr * (2 * diff * Q[j] - 2 * reg * P[i])
                Q[j] += lr * (2 * diff * P[i] - 2 * reg * Q[j])
                BP[i] += lr * (2 * diff - 2 * reg * BP[i])
                BQ[j] += lr * (2 * diff - 2 * reg * BQ[j])

    return P, Q, B, BP, BQ

`build_R_hat` 函数构建评分预测矩阵 $\hat{R}$。

用户 i 对电影 j 的评分为 $ r_{ij} = p_iq_j^T + b + bp_i + bq_j$。

In [ ]:
def build_R_hat(P: np.ndarray, Q: np.ndarray, B: float, BP: np.ndarray, BQ: np.ndarray):
    m = P.shape[0]
    n = Q.shape[0]

    return (
        P @ Q.T
        + B
        + BP.reshape((-1, 1)).repeat(n, axis=1)
        + BQ.reshape((1, -1)).repeat(m, axis=0)
    )

`evaluate` 函数在真实的评分数据集上，使用 MSE 来评估预测的准确性。

In [ ]:
def evaluate(R_hat: np.ndarray, R_real: np.ndarray):
    m, n = R_real.shape

    values_num = np.count_nonzero(~np.isnan(R_real))
    square_error = 0.0

    for i in range(m):
        for j in range(n):
            if np.isnan(R_real[i, j]):
                continue

            square_error += (R_real[i, j] - R_hat[i, j]) ** 2

    return square_error / values_num

使用训练集计算评分预测矩阵。

In [ ]:
R_train = build_R(train_set)
P, Q, B, BP, BQ = matrix_factorization(R_train)
R_hat = build_R_hat(P, Q, B, BP, BQ)

使用验证集调整超参数。

In [ ]:
R_validation = build_R(validation_set)
mse = evaluate(R_hat, R_validation)
print("MSE on validation set: ", mse)

使用测试集评估模型的准确性。

In [ ]:
R_test = build_R(test_set)
mse = evaluate(R_hat, R_test)
print("MSE on test set: ", mse)